<img src = 'https://chita-brita.ru/wp-content/uploads/2021/10/dmp.jpg'>

# <center> Система рекомендаций фильмов на основе матричной факторизации.

**Цель проекта** - составить систему рекомендаций фильмов на основе матричной факторизации.

Нам предоставили два набора данных:
1. Данные по фильмам
2. Данные по рейтингам, выставленным пользователями.

In [10]:
import numpy as np
import scipy
import pandas as pd

from implicit.als import AlternatingLeastSquares
from implicit.evaluation import mean_average_precision_at_k
from lightfm import LightFM
from lightfm.evaluation import precision_at_k

from sklearn.model_selection import train_test_split

c:\Users\Олег\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


Загрузим наш датасет с оценками, которые выставляли пользователи фильмам, добавим имена столбцам, затем отсортируем записи по дате и преобразуем целевую переменную в бинарный вид: условимся, что фильмы, которым пользователь поставил оценку выше 2, считаются понравившимися ему (класс 1), в противном случае — не понравившимися (класс 0):

In [2]:
ratings = pd.read_csv("ml-100k/u.data", sep="\t", header=None)
ratings.columns = ['user_id', 'item_id', 'rating', 'timestamp']
ratings.sort_values('timestamp', inplace=True)
ratings['score'] = (ratings['rating'] > 2).apply(int)
ratings

,user_id,item_id,rating,timestamp,score
214,259,255,4,874724710,1
83965,259,286,4,874724727,1
43027,259,298,4,874724754,1
21396,259,185,4,874724781,1
82655,259,173,4,874724843,1
...,...,...,...,...,...
46773,729,689,4,893286638,1
73008,729,313,3,893286638,1
46574,729,328,3,893286638,1
64312,729,748,4,893286638,1


Разделим нашу выборку на тренировочную и тестовую в соотношении 80/20 без перемешивания. В тренировочную выборку попадут оценки пользователей за первые 80 % периода проведения наблюдений, а в тестовую — оставшиеся 20 %:

In [3]:
train, test = train_test_split(ratings, test_size=0.2, shuffle=False)

Создадим user-item таблицу для тренировочной и тестовой выборки. В этой таблице по строкам - идентификаторы всех уникальных пользователей, которые у нас есть, а по столбцам — все уникальные фильмы. Получаем две матрицы размерности 943 x 1682. На пересечении строк и столбцов этих матриц - числа, характеризующие наличие положительных и отрицательных оценок пользователей.

In [4]:
# Посмотрим на количество уникальных пользователей, создав сводные таблицы для тренировочной и тестовой выборок. 
train_pivot = pd.pivot_table(
    train,
    index="user_id", 
    columns="item_id", 
    values="score"
)
test_pivot = pd.pivot_table(
    test,
    index="user_id", 
    columns="item_id", 
    values="score"
)

print(train_pivot.shape)
print(test_pivot.shape)

(751, 1616)
(301, 1448)


Видим, что в тренировочную user-item таблицу попали оценки от 751 пользователей для 1616 товаров, а в тестовую — от 301 пользователей для 1448 товаров. 

Теперь создадим сводную таблицу из таблицы rating, заполнив её ячейки нулями. Получим матрицу размером 943 x 1682. Для тех фильмов, которым пользователь выставил оценку значения, будут равны 0, для остальных — пропуску. 

In [5]:
shell = pd.pivot_table(
    ratings, 
    index="user_id", 
    columns="item_id", 
    values="score", 
    aggfunc=lambda x: 0
)
shell.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Чтобы получить тренировочную и тестовую user-item таблицы, сложим таблицу shell с соответствующими таблицами train_pivot и test_pivot.

Чтобы корректно обрабатывать пропущенные значения, мы трансформируем 1 в 2, а 0 — в 1. Сами пропуски заполняем нулями. В результате у нас получатся две таблицы размером 943 x 1682, в которых на пересечении пользователя и фильма стоит:
* 0 — если пользователь не оценил данный фильм;
* 1 — если пользователь оценил фильм отрицательно;
* 2 — если пользователь оценил фильм положительно.

In [6]:
train_pivot = shell + train_pivot
test_pivot = shell + test_pivot

train_pivot = (train_pivot + 1).fillna(0)
test_pivot = (test_pivot + 1).fillna(0)
print(train_pivot.shape)
print(test_pivot.shape)
## (943, 1682)
## (943, 1682)

train_pivot.head()

(943, 1682)
(943, 1682)


item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,2.0,2.0,2.0,2.0,0.0,2.0,2.0,1.0,2.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Поскольку модели из библиотеки implicit требуют, чтобы user-item матрицы были представлены в виде разреженных матриц, используем функцию csr_matrix() из модуля sparse библиотеки scipy:

In [7]:
train_pivot_sparse = scipy.sparse.csr_matrix(train_pivot.values)
test_pivot_sparse = scipy.sparse.csr_matrix(test_pivot.values)

Обучим ALS-модель с 10-ю факторами, параметр random_state установим в значение 42.

In [8]:
model = AlternatingLeastSquares(factors=10, random_state=42)
model.fit(train_pivot_sparse)

c:\Users\Олег\AppData\Local\Programs\Python\Python311\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/15 [00:00<?, ?it/s]

Определим качество модели на всей тестовой выборке, рассчитав precision для топ 10-рекомендуемых фильмов с помощью функции mean_average_precision_at_k() из библиотеки implicit:

In [9]:
map_at10 = mean_average_precision_at_k(model, train_pivot_sparse, test_pivot_sparse, K=10)
print('Mean Average Precision at 10: {:.3f}'.format(map_at10))

  0%|          | 0/301 [00:00<?, ?it/s]

Mean Average Precision at 10: 0.084


Попробуем улучшить качество рекомендаций, воспользовавшись факторизационными машинами. 

Обучим факторизационные машины с 10-ю факторами, в качестве функции потерь используем logloss, параметр random_state установим в значение 42. Обучение будем производить на 30 итерациях (эпохах):

In [11]:
model = LightFM(no_components=10, loss='logistic', random_state=42)
model.fit(train_pivot_sparse, epochs=30)

Рассчитаем средний precision для топ 10 рекомендуемых фильмов по всей тестовой выборке.

In [12]:
map_at10 = precision_at_k(model, test_pivot_sparse, k=10).mean()
print('Mean Average Precision at 10: {:.2f}'.format(map_at10))

Mean Average Precision at 10: 0.32


Итак, наша метрика составила 0.32, что гораздо выше показателя, полученного с помощью ALS.

**Цель проекта выполнена**.